In this notebook, we show some supporting functions for reading, visualizing the training and testing data. And, we give a demo on how to use the provided supporting functions to predict the labels for each motor and produce outputs in the required format for the data challenge submission. Finally, we show a demo on how to prepare the submission file based on the provided csv template.

## Reading the data.

As we shown in WP_1, we can use the `read_all_tst_data_from_path` function to read and visualize data. Please note you have the option to define the preprocessing you would like to do on the original data. For this, you just need to define your preprocessing function and pass its function handle to the `read_all_tst_data_from_path` function.

Below is a sample code that reads and visualize all the training dataset, where we apply a simple outlier removal based on validity range as pre-processing. We also remove sequence-to-sequence variablity in the pre-processing. Please note that you need to download the datasets `training_data.zip` and `testing_data.zip` from [kaggle](https://www.kaggle.com/competitions/robot-predictive-maintenance/data) and unzip them. You need to change the path in the sample code below to the path of your datasets.

### Training data.

In [2]:
# Ignore warnings.
warnings.filterwarnings('ignore')

# Read all the dataset. Change to your dictionary if needed.
base_dictionary = '../../dataset/training_data/'
df_train = read_all_test_data_from_path(base_dictionary, pre_processing, is_plot=False)

### Testing data.

In [3]:
# Read all the dataset. Change to your dictionary if needed.
base_dictionary = '../../dataset/testing_data/'
df_test = read_all_test_data_from_path(base_dictionary, pre_processing, is_plot=False)

## Demo: Apply a classification-based fault detection model.

In this section, we use motor $6$ as an example to demonstrate how to train a classification-based fault detection model and apply it to predict the labels on the testing dataset. The basic steps are the same as Workpackage 2. However, you need to pay attention if you use sliding windows to augument the feature space. In this case, the first window_size points in each sequence were scaped in the augumented feature space, as we do not have enough points in the history. In the data challenge, these scaped points need to be predicted manually, as the evaluation is done on all the points, including the scaped points.

In the current version of `prepare_sliding_window`, we addressed this issue by filling the missed history based on the closest available observations in the dataset. Therefore, you just need to make sure you use this version of `utility.py`, the scaped points will be filled automatically.

Below, you can find a demo how to train a logistic regression model to predict the labels for motor 6. For more details on the classification-bsaed fault detection models, you can have a look at the tutorials in [WP2](../../supporting_scripts/WP_2_20240516/demo_motor_6_lr.ipynb).

### Training

In [4]:
from utility import extract_selected_feature, prepare_sliding_window

# Define the motor index.
motor_idx = 6

# Specify the test conditions you would like to include in the training.
df_data_experiment = df_train[df_train['test_condition'].isin(['20240425_093699', '20240425_094425', '20240426_140055',
                                                       '20240503_164675', '20240503_165189',
                                                       '20240503_163963', '20240325_155003'])]

# Define the features.
feature_list_all = ['time', 'data_motor_1_position', 'data_motor_1_temperature', 'data_motor_1_voltage',
                    'data_motor_2_position', 'data_motor_2_temperature', 'data_motor_2_voltage',
                    'data_motor_3_position', 'data_motor_3_temperature', 'data_motor_3_voltage',
                    'data_motor_4_position', 'data_motor_4_temperature', 'data_motor_4_voltage',
                    'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_5_voltage',
                    'data_motor_6_position', 'data_motor_6_temperature', 'data_motor_6_voltage']
# Extract the features.
df_tr_x, df_tr_y = extract_selected_feature(df_data_experiment, feature_list_all, motor_idx, mdl_type='clf')

# Prepare the training data based on the defined sliding window.
window_size = 50
sample_step = 10
X_train, y_train = prepare_sliding_window(df_x=df_tr_x, y=df_tr_y, window_size=window_size, sample_step=sample_step, mdl_type='clf')

# Define the classification model.
# Define the steps of the pipeline
steps = [
    ('standardizer', StandardScaler()),  # Step 1: StandardScaler
    ('mdl', LogisticRegression(class_weight='balanced'))    # Step 2: Linear Regression
]

# Create the pipeline
pipeline = Pipeline(steps)

# Define hyperparameters to search
param_grid = {
    'mdl__C': [0.001, 0.01, 0.1, 1, 10, 100]  # Inverse of regularization strength
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='f1', cv=5)

# Train the model.
mdl = grid_search.fit(X_train, y_train)

### Make prediction on the testing dataset.

In [6]:
# Prepare for the testing dataset.
# Define the features.
feature_list_all = ['time', 'data_motor_1_position', 'data_motor_1_temperature', 'data_motor_1_voltage',
                    'data_motor_2_position', 'data_motor_2_temperature', 'data_motor_2_voltage',
                    'data_motor_3_position', 'data_motor_3_temperature', 'data_motor_3_voltage',
                    'data_motor_4_position', 'data_motor_4_temperature', 'data_motor_4_voltage',
                    'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_5_voltage',
                    'data_motor_6_position', 'data_motor_6_temperature', 'data_motor_6_voltage']
# Add test_condition for extracting different sequences.
feature_list_all.append('test_condition')
# Get the features.
df_test_x = df_test[feature_list_all]
# Augument the features in the same way as the training data.
X_test = prepare_sliding_window(df_x=df_test_x, window_size=window_size, sample_step=sample_step, mdl_type='clf')
# Make prediction.
y_pred_6 = grid_search.predict(X_test)

## Demo: Apply a regression-based fault detection model.

In this section, we will apply a regression-based fault detection model to the data. We choose the motor $5$ as an example. For details on regression-based fault detection, please refer to the tutorials in [WP3](../../supporting_scripts/WP_3_20240521/demo_FaultDetectReg.ipynb).

### Training the regression model.

In [7]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from utility import read_all_test_data_from_path, extract_selected_feature, prepare_sliding_window, FaultDetectReg

# Pre-train the model.
# Get all the normal data for motor 5.
normal_test_id = ['20240105_164214',
    '20240105_165300',
    '20240105_165972',
    '20240320_152031',
    '20240320_153841',
    '20240320_155664',
    '20240321_122650',
    '20240325_135213',
    '20240325_152902',
    '20240426_141190',
    '20240426_141532',
    '20240426_141602',
    '20240426_141726',
    '20240426_141938',
    '20240426_141980',
    '20240503_163963',
    '20240503_164435',
    '20240503_164675',
    '20240503_165189'
]

df_tr_motor_5 = df_train[df_train['test_condition'].isin(normal_test_id)]

feature_list_all = ['time', 'data_motor_1_position', 'data_motor_1_temperature', 'data_motor_1_voltage',
                'data_motor_2_position', 'data_motor_2_temperature', 'data_motor_2_voltage',
                'data_motor_3_position', 'data_motor_3_temperature', 'data_motor_3_voltage',
                'data_motor_4_position', 'data_motor_4_temperature', 'data_motor_4_voltage',
                'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_5_voltage',
                'data_motor_6_position', 'data_motor_6_temperature', 'data_motor_6_voltage']

# Prepare feature and response of the training dataset.
x_tr_org, y_temp_tr_org = extract_selected_feature(df_data=df_tr_motor_5, feature_list=feature_list_all, motor_idx=5, mdl_type='reg')

# Enrich the features based on the sliding window.
window_size = 10
sample_step = 1
prediction_lead_time = 1 
threshold = .9
abnormal_limit = 3

x_tr, y_temp_tr = prepare_sliding_window(df_x=x_tr_org, y=y_temp_tr_org, window_size=window_size, sample_step=sample_step, prediction_lead_time=prediction_lead_time, mdl_type='reg')

# Define the steps of the pipeline
steps = [
    ('standardizer', StandardScaler()),  # Step 1: StandardScaler
    ('regressor', LinearRegression())    # Step 2: Linear Regression
]

# Create the pipeline
mdl_linear_regreession = Pipeline(steps)
# Fit the model
mdl = mdl_linear_regreession.fit(x_tr, y_temp_tr)

### Making prediction.

In [10]:
# Define the fault detector.
detector_reg = FaultDetectReg(reg_mdl=mdl, threshold=threshold, abnormal_limit=abnormal_limit, window_size=window_size, sample_step=sample_step, pred_lead_time=prediction_lead_time)

# Define the features.
feature_list_all = ['time', 'data_motor_1_position', 'data_motor_1_temperature', 'data_motor_1_voltage',
                    'data_motor_2_position', 'data_motor_2_temperature', 'data_motor_2_voltage',
                    'data_motor_3_position', 'data_motor_3_temperature', 'data_motor_3_voltage',
                    'data_motor_4_position', 'data_motor_4_temperature', 'data_motor_4_voltage',
                    'data_motor_5_position', 'data_motor_5_temperature', 'data_motor_5_voltage',
                    'data_motor_6_position', 'data_motor_6_temperature', 'data_motor_6_voltage']
# Prepare the testing data.
x_test_org, y_temp_test_org = extract_selected_feature(df_data=df_test, feature_list=feature_list_all, motor_idx=5, mdl_type='reg')

# Make predicition.
y_pred_5, y_response_test_pred = detector_reg.predict(df_x_test=x_test_org, y_response_test=y_temp_test_org, complement_truncation=True)

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:30<00:00,  3.87s/it]


## Prepare the results as a submission file for the data challenge.

In this section, we demo how to prepare the results as a submission file for the data challenge. First, we need to download the submission template `sample_submission.csv` from [kaggle](https://www.kaggle.com/competitions/robot-predictive-maintenance/data). As shown below, in this csv files, we just need to give our prediction on the six motors in the corresponding columns. You can find a demo below.

In [17]:
import pandas as pd

# Read the template.
path = 'kaggle_data_challenge/sample_submission.csv' # Change to your path.
df_submission = pd.read_csv(path)

# Initial all values with -1.
df_submission.loc[:, ['data_motor_1_label', 'data_motor_2_label', 'data_motor_3_label', 'data_motor_4_label', 'data_motor_5_label', 'data_motor_6_label']] = -1
df_submission.head()

,idx,data_motor_1_label,data_motor_2_label,data_motor_3_label,data_motor_4_label,data_motor_5_label,data_motor_6_label,test_condition
0,0,-1,-1,-1,-1,-1,-1,20240527_094865
1,1,-1,-1,-1,-1,-1,-1,20240527_094865
2,2,-1,-1,-1,-1,-1,-1,20240527_094865
3,3,-1,-1,-1,-1,-1,-1,20240527_094865
4,4,-1,-1,-1,-1,-1,-1,20240527_094865


In [18]:
# Replace each column with your prediction.
df_submission['data_motor_5_label'] = y_pred_5
df_submission['data_motor_6_label'] = y_pred_6

# For the other motors, we just fill with 0.
df_submission.loc[:, ['data_motor_1_label', 'data_motor_2_label', 'data_motor_3_label', 'data_motor_4_label']] = 0

In [19]:
# Output the submission csv.
df_submission.to_csv('../ws_prepare_data_challenge/submission.csv')